In [10]:
import psycopg2
import pandas as pd
from datetime import time, datetime as dt
from dateutil.relativedelta import relativedelta

In [2]:
company_id = 'contato_4b42bcf5-6cb5-47d7-8a4e-6c14b30f7dac'

In [6]:
df_sales = pd.read_excel(r"C:\Users\joaopedro\Downloads\dados_app_contato.xlsx", sheet_name='Vendas')

In [3]:
df_rh = pd.read_excel(r"C:\Users\joaopedro\Downloads\dados_app_contato.xlsx", sheet_name='rh')
df_projeto = pd.read_excel(r"C:\Users\joaopedro\Downloads\dados_app_contato.xlsx", sheet_name='projeto')
df_adm = pd.read_excel(r"C:\Users\joaopedro\Downloads\dados_app_contato.xlsx", sheet_name='adm')
df_produtos = pd.read_excel(r"C:\Users\joaopedro\Downloads\dados_app_contato.xlsx", sheet_name='produtos')
df_comissao = pd.read_excel(r"C:\Users\joaopedro\Downloads\dados_app_contato.xlsx", sheet_name='comissao')
df_receita = pd.read_excel(r"C:\Users\joaopedro\Downloads\dados_app_contato.xlsx", sheet_name='receita')

In [14]:
df_comissao.to_excel(r"C:\Users\joaopedro\Downloads\app_contato_comissao.xlsx", index=False)
df_receita.to_excel(r"C:\Users\joaopedro\Downloads\app_contato_receita.xlsx", index=False)

In [20]:
conn = psycopg2.connect(dbname='postgres', user='postgres', password='1673', host='localhost', port='5432')
cur = conn.cursor()

In [3]:
cur.execute("select * from sales;")
name_columns = [x[0] for x in cur.description]
df_sales = pd.DataFrame(cur.fetchall(), columns=name_columns)

In [5]:
cur.execute("select id_income, qtd_parcelas from income;")
name_columns = [x[0] for x in cur.description]
df_income = pd.DataFrame(cur.fetchall(), columns=name_columns)

In [7]:
df_sales_upd = pd.merge(df_sales, df_income, on='id_income', how='left')

In [22]:
lista_data = []
for row in df_sales_upd.itertuples():
    data = row.data_primeira_parcela
    qtd_parcelas_receita = row.qtd_parcelas
    data_ultima_parcela_income = data + relativedelta(months=(qtd_parcelas_receita-1))
    data_ultima_parcela_income_upd = dt(data_ultima_parcela_income.year, data_ultima_parcela_income.month, 10).date()

    query = f"UPDATE sales SET data_ultima_parcela_receita = '{data_ultima_parcela_income_upd}' WHERE id_venda = '{row.id_venda}';"
    cur.execute(query)
conn.commit()

In [18]:
df_sales['id_venda'].drop_duplicates()

0         210332
1         210333
2      100086837
3      100086835
4        2101890
         ...    
592     20407701
593    400517903
594     26068108
595    400517915
596    400517911
Name: id_venda, Length: 597, dtype: object

In [13]:
df_sales_upd

,id_prod_caract,nome_adm,nome_produto,id_funcionario,nome_funcionario,cota,projeto,nome_cliente,data_venda,data_venda_abv,...,taxa_comissao,valor_comissao,company_id,taxa_receita,valor_receita,id_income,grupo,id_venda,data_ultima_parcela_receita,qtd_parcelas
0,205,Groscon,GSC + 1Im200,1,Andyara,346,Network,PAULA DANGELO LIMA PADILHA,2023-01-11,2023/01,...,1.5000000000000002,5046.768750000001,contato_4b42bcf5-6cb5-47d7-8a4e-6c14b30f7dac,5.0,16822.5625,31,56,210332,2024-01-10,11
1,205,Groscon,GSC + 1Im200,1,Andyara,608,Network,PAULA DANGELO LIMA PADILHA,2023-01-11,2023/01,...,1.5000000000000002,5046.768750000001,contato_4b42bcf5-6cb5-47d7-8a4e-6c14b30f7dac,5.0,16822.5625,31,56,210333,2024-01-10,11
2,74,Âncora,ANC - Fácil,3,Roberto Andrade,927,Especial 2,PAULO HENRIQUE MUBUCCI MIRANDA,2023-01-12,2023/01,...,1.6999999999999995,1054.0,contato_4b42bcf5-6cb5-47d7-8a4e-6c14b30f7dac,4.0,2480.0,11,523,100086837,2023-10-10,8
3,74,Âncora,ANC - Fácil,3,Roberto Andrade,900,Especial 2,PAULO HENRIQUE MUBUCCI MIRANDA,2023-01-12,2023/01,...,1.6999999999999995,1054.0,contato_4b42bcf5-6cb5-47d7-8a4e-6c14b30f7dac,4.0,2480.0,11,523,100086835,2023-10-10,8
4,43,Magalu,MGL - Auto/Moto/Eletro/Serv Linear,13,Camila,75,Externo,FLAVIO DE OLIVEIRA SANTOS,2023-01-12,2023/01,...,0.7000000000000001,25.9,contato_4b42bcf5-6cb5-47d7-8a4e-6c14b30f7dac,2.8,103.6,7,4142,2101890,2023-08-10,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
592,240,Yamaha,YAM Moto,2,Beto Bussola,403,Network,GUIDO MESSIAS ALBARELO,2024-05-27,2024/05,...,1.5,574.635,contato_4b42bcf5-6cb5-47d7-8a4e-6c14b30f7dac,5.5,2106.995,36,9002,20407701,2024-11-10,5
593,2,BR,BR - Im/Auto,1,Andyara,907,Network,SUMER COMERCIO DE PNEUS LTDA,2024-05-28,2024/05,...,1.5,1350.0,contato_4b42bcf5-6cb5-47d7-8a4e-6c14b30f7dac,3.0000000000000004,2700.0,1,1052,400517903,2024-09-10,3
594,0,Servopa,SVP Imóvel B50,0,Não registrado,5110,0,VINICIUS FRANCO MACHADO,2024-05-29,2024/05,...,0.0,0.0,contato_4b42bcf5-6cb5-47d7-8a4e-6c14b30f7dac,1.7500000000000002,1771.63,33,1554,26068108,2025-05-10,10
595,2,BR,BR - Im/Auto,32,Pedro Dalla,234,Network,TIAGO NUNES LOPES,2024-05-29,2024/05,...,1.5,2025.0,contato_4b42bcf5-6cb5-47d7-8a4e-6c14b30f7dac,3.0000000000000004,4050.000000000001,1,5033,400517915,2024-09-10,3


In [19]:
for row in df_projeto.itertuples():
    query = f"INSERT INTO projects (id_projeto, nome_projeto, company_id) VALUES ({row.id_projeto}, '{row.nome_projeto}', '{company_id}');"
    cur.execute(query)
conn.commit()

In [20]:
for row in df_rh[~pd.isna(df_rh['pix'])].itertuples():
    # query = f"INSERT INTO employees (id_funcionario, nome_funcionario, cpf, data_adesao, salario, pix, projeto, emp_validation, company_id) VALUES ({row.id_funcionario}, '{row.nome_funcionario}', '{row.cpf}', \
    #     '{row.data_adesao}', {salario}, '{pix}', true, '{company_id}');"
    query = f"INSERT INTO employees (id_funcionario, nome_funcionario, pix) VALUES ({row.id_funcionario}, '{row.nome_funcionario}', '{row.pix}');"
    cur.execute(query)
conn.commit()

In [21]:
for row in df_adm.itertuples():
    query = f"INSERT INTO administrators (id_adm, nome_adm, company_id) VALUES ({row.id_adm}, '{row.nome_adm}', '{company_id}');"
    cur.execute(query)
conn.commit()

In [22]:
for row in df_produtos.itertuples():
    query = f"INSERT INTO products (id_produto, nome_adm, nome_produto, company_id) VALUES ({row.id_produto}, '{row.nome_adm}', '{row.nome_produto}', '{company_id}');"
    cur.execute(query)
conn.commit()

In [25]:
def combine_columns(row):
    list_cols = range(1,15)
    list_value = []
    for col in list_cols:
        if pd.isna(row[col]) == False:
            value = round(row[col]*100,4)
            list_value.append(str(value))
            list_value.append(' - ')
    final_value = ''.join(list_value[:-1])
    return final_value

# Apply the custom function to create a new column 'Combined'
df_comissao['taxa_parcelas'] = df_comissao.apply(combine_columns, axis=1)
df_receita['taxa_parcelas'] = df_receita.apply(combine_columns, axis=1)

In [31]:
df_comissao['taxa_comissao'] = df_comissao['taxa_comissao']*100
df_receita['taxa_comissao'] = df_receita['taxa_comissao']*100

In [32]:
for row in df_receita.itertuples():
    query = f"INSERT INTO income (id_income, nome_adm, nome_produto, taxa_comissao, contemplacao, qtd_parcelas, taxa_parcelas, company_id) VALUES ({row.id_income}, '{row.nome_adm}', '{row.nome_produto}', \
        {row.taxa_comissao}, {row.contemplacao}, {row.qtd_parcelas}, '{row.taxa_parcelas}', '{company_id}');"
    cur.execute(query)
conn.commit()

In [33]:
for row in df_comissao.itertuples():
    query = f"INSERT INTO prod_caract (id_prod_caract, nome_adm, nome_produto, projeto, taxa_comissao, qtd_parcelas, taxa_parcelas, company_id) VALUES ({row.id_prod_caract}, '{row.nome_adm}', '{row.nome_produto}', '{row.projeto}', \
        {row.taxa_comissao}, {row.qtd_parcelas}, '{row.taxa_parcelas}', '{company_id}');"
    cur.execute(query)
conn.commit()

In [ ]:
id_prod_caract, projeto, data_primeira_parcela, data_ultima_parcela, valor_venda, taxa_comissao, valor_comissao, taxa_receita, valor_receita, id_income

In [52]:
cur.execute("select nome_funcionario from employees;")
list_func = list(cur.fetchall())

list_func = [x[0] for x in list_func]

In [53]:
set(df_sales['nome_funcionario']) - set(list_func)

{None}

In [35]:
cur.execute("select nome_adm, nome_produto from products;")
df_prod = pd.DataFrame(cur.fetchall(), columns=['nome_adm','nome_produto'])

In [37]:
df_sales = pd.merge(df_sales, df_prod, on='nome_produto', how='left')

In [40]:
df_sales['data_venda'] = pd.to_datetime(df_sales['data_venda'])
df_sales['data_venda_abv'] = df_sales.data_venda.map(lambda x: f'{x.year}/0{x.month}' if len(str(x.month)) == 1 else f'{x.year}/{x.month}')

In [8]:
df_sales.loc[df_sales['nome_funcionario'] == 'ROBERTO BUSSOLA', 'id_funcionario'] = 2
df_sales.loc[df_sales['nome_funcionario'] == 'ANDYARA GUSMAN', 'id_funcionario'] = 1
df_sales.loc[df_sales['nome_funcionario'] == 'DANIEL BUSSOLA', 'id_funcionario'] = 5
df_sales.loc[df_sales['nome_funcionario'] == 'Roberto T', 'id_funcionario'] = 24
df_sales.loc[df_sales['nome_funcionario'] == 'ROBERTO ANDRADE', 'id_funcionario'] = 3
df_sales.loc[df_sales['nome_funcionario'] == 'FLAVIO OLIVEIRA', 'id_funcionario'] = 17
df_sales.loc[df_sales['nome_funcionario'] == 'Carlos', 'id_funcionario'] = 29
df_sales.loc[df_sales['nome_funcionario'] == 'Tabata', 'id_funcionario'] = 16
df_sales.loc[df_sales['nome_funcionario'] == 'Isis', 'id_funcionario'] = 9
df_sales.loc[df_sales['nome_funcionario'] == 'REGIANE BIGON', 'id_funcionario'] = 11
df_sales.loc[df_sales['nome_funcionario'] == 'Eduardo/Thaís', 'id_funcionario'] = 21
df_sales.loc[df_sales['nome_funcionario'] == 'JOÃO NETO', 'id_funcionario'] = 34
df_sales.loc[df_sales['nome_funcionario'] == 'PEDRO DALLA', 'id_funcionario'] = 32
df_sales.loc[df_sales['nome_funcionario'] == 'YAGO GOBBE', 'id_funcionario'] = 33
df_sales.loc[df_sales['nome_funcionario'] == 'Gabrielle', 'id_funcionario'] = 26
df_sales.loc[df_sales['nome_funcionario'] == 'DANIELA FARIA', 'id_funcionario'] = 35
df_sales.loc[df_sales['nome_funcionario'] == 'Rogério', 'id_funcionario'] = 28
df_sales.loc[df_sales['nome_funcionario'] == 'Renato', 'id_funcionario'] = 12
df_sales.loc[df_sales['nome_funcionario'] == 'Giovani C.', 'id_funcionario'] = 25
df_sales.loc[df_sales['nome_funcionario'] == 'LUCIANO HONORIO', 'id_funcionario'] = 31
df_sales.loc[df_sales['nome_funcionario'] == 'Marcos', 'id_funcionario'] = 27
df_sales.loc[df_sales['nome_funcionario'] == 'EVANDRO MOURA', 'id_funcionario'] = 38
df_sales.loc[df_sales['nome_funcionario'] == 'MERCIA', 'id_funcionario'] = 41
df_sales.loc[df_sales['nome_funcionario'] == 'LEONARDO TIROLI', 'id_funcionario'] = 40
df_sales.loc[df_sales['nome_funcionario'] == 'KAREN GONÇALVES', 'id_funcionario'] = 43
df_sales.loc[df_sales['nome_funcionario'] == 'Mercia', 'id_funcionario'] = 41
df_sales.loc[df_sales['nome_funcionario'] == 'Gustavo', 'id_funcionario'] = 36
df_sales.loc[df_sales['nome_funcionario'] == 'TORO', 'id_funcionario'] = 39
df_sales.loc[df_sales['nome_funcionario'] == 'Camila', 'id_funcionario'] = 13
df_sales.loc[df_sales['nome_funcionario'] == 'Flávio Martorelli', 'id_funcionario'] = 8
df_sales.loc[df_sales['nome_funcionario'] == 'LUCAS LOPES', 'id_funcionario'] = 44

In [12]:
df_sales['id_funcionario'] = df_sales['id_funcionario'].fillna(0)
df_sales['id_funcionario'] = df_sales['id_funcionario'].map(int)

# Revisitar os IDs dos funcionários pelos nomes

In [43]:
list_nome_funcionario = []
for row in df_sales.itertuples():
    if row.id_funcionario != 0:
        nome_funcionario = df_rh.loc[df_rh['id_funcionario'] == row.id_funcionario, 'nome_funcionario'].values[0]
        list_nome_funcionario.append(nome_funcionario)
    else:
        list_nome_funcionario.append(None)
df_sales['nome_funcionario'] = list_nome_funcionario